# Quasidynamic Physics Simulator for Contact Implicit MPC

In this notebook we will be writing a Quasdynamic Physics Simulator from scratch. Given a set of spherical collision bodies for a robot, and a sphere to manipulate, the goal is to compute one physics step for the next set of states and contact forces that each collision body experiences. This will be following the derivation outlined in https://arxiv.org/pdf/2505.02291 which should be read for more details. The advantage of a quasidynamic simulator is it doesn't require a state for velocity and greatly simplifies forward physics simulation. A quasidynamic model can simulate further ahead in time with a larger timestep than a second-order simulator by ignoring transient dynamics and focusing on transitions between steady states.

The physics simulator works by framing the physics step as an optimization problem, and computing the optimal solution for the object and robot states as well as all the contact forces. The KKT conditions for the optimization problem are equivalent to the force balance equations of a quasidynamic system. 

Our simulator will work directly in the generalized coordinates $q$. Where $q^a$ are the coordinates for the robot (the joint values) and $q^o$ are the coordinates of the object (x, y, z).
At a configuration $q$, the equations of motion are framed as a search for next configurations $q_+ = (q_+^a, q_+^o)$ such that the following force balance equations hold,

\begin{align}
\mathbf{K}_a (q_+^a - u) &= \tau^a + \sum_{i=1}^{n_c} (\mathbf{J}_{a_i}(q))^\top \lambda_i \tag{1a} \\
\epsilon \mathbf{M}_o(q) \frac{q_+^o - q^o}{h^2} &= \tau^o + \sum_{i=1}^{n_c} (\mathbf{J}_{o_i}(q))^\top \lambda_i \tag{1b}
\end{align}

where $h \in \mathbb{R}_{>0}$ is the timestep, 
$\tau^a \in \mathbb{R}^{n_{q_a}}$ and $\tau^o \in \mathbb{R}^{n_{q_o}}$ 
are external torques acting on actuated and unactuated bodies respectively (e.g., gravity), and $\epsilon \in \mathbb{R}_{\ge 0}$ is a regularization constant. These equations are simply a force balance.

We can then stack the Jacobians and write a matrix representation of the above two equations into one equation:

\begin{equation}
\mathbf{J}_i := [\mathbf{J}_{o_i}, \mathbf{J}_{a_i}] :=
\begin{bmatrix}
\mathbf{J}_{n_i} \\
\mathbf{J}_{t_i}
\end{bmatrix}
\in \mathbb{R}^{3 \times n_q}.
\tag{2}
\end{equation}

\begin{equation}
\mathbf{P}(q) q_+ + \mathbf{b}(q, u)
- \sum_{i=1}^{n_c} \mathbf{J}_i(q)^\top \lambda_i = 0,
\tag{3a}
\end{equation}
where
\begin{align}
\mathbf{P}(q) &:= 
\begin{bmatrix}
\epsilon \mathbf{M}_o(q) / h^2 & 0 \\
0 & \mathbf{K}_a
\end{bmatrix}, \tag{3b} \\
\mathbf{b}(q, u) &:= -
\begin{bmatrix}
\epsilon \mathbf{M}_o(q) q^o / h^2 + \tau^o \\
\mathbf{K}_a u + \tau^a
\end{bmatrix}
\tag{3c}
\end{align}

\begin{equation}
\mathbf{c}_i(q) := [\phi_i(q), 0, 0]^\top - \mathbf{J}_i(q) q
\end{equation}


If we solve the following optimization problem using a log barrier relaxation for the friction cone constraints, then the solution is the coordinates for the next time step:

\begin{equation}
\min_{q_+} \;
\frac{1}{2} q_+^\top \mathbf{P} q_+ + \mathbf{b}^\top q_+
- \frac{1}{\kappa} \sum_{i=1}^{n_c} 
\log \left( \frac{\nu_{n_i}^2}{\mu_i^2} - \| \nu_{t_i} \|^2 \right),
\tag{10}
\end{equation}

where $\nu_i = \mathbf{J}_i q_+ + \mathbf{c}_i$, and 
$\nu_i = (\nu_{n_i}, \nu_{t_i})$ with 
$\nu_{n_i} \in \mathbb{R}$ and 
$\nu_{t_i} \in \mathbb{R}^2$. 
The resulting dynamics creates a force-field effect where bodies that are not in contact apply forces inversely proportional to their distance.

Furthermore, the following equation gives the average contact force over the timestep:

\begin{equation}
\lambda_{\kappa,i} =
\frac{2 \kappa^{-1}}{\nu_{n_i}^2 / \mu_i^2 - \| \nu_{t_i} \|^2}
\begin{bmatrix}
\nu_{n_i} / \mu_i^2 \\
-\nu_{t_i}
\end{bmatrix}.
\tag{12}
\end{equation}

This contact model is differentiable and can allow large timesteps to be used without breaking, thus making it ideal for MPC.


### Outline

1. Use MuJoCo to load the scene with a dexterous robot hand and a ball. 
2. Setup Viser to do visualization of contact bodies and contact forces. 
3. Setup Pinnocchio for inverse kinematics.
4. Configure spheres to represent collision bodies. 
5. Compute contacts, contact Jacobians, and the signed distance function for each collision body. 
6. Compute the matrices used in the optimization problem 
7. Solve the optimization problem using IPOPT and compute the contact forces

### 1.

In [23]:
urdf_path = "assets/scene/robotis_5F_hand/default.urdf"
mesh_path = "assets/scene/robotis_5F_hand"
env_path = "assets/scene/robotis_5F_hand/default.xml"

default_q = [0.1, 0, -1.5, 0.2, 0.2, -0.1, 0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2, 0.1, 0.2, 0.2, 0.2]
n_joints = len(default_q)
table_height = -0.2 

### 2. 

Initialize a Viser server and load the URDF of the robot at the default configuration. 
Open the URL in the browser to see the visualizations.

In [ ]:
#create Viser server for visualiztion 
 
import time
import numpy as np
import pinocchio as pin
from yourdfpy import URDF
import viser 

from viser.extras import ViserUrdf


viser_server = viser.ViserServer()     

urdf = URDF.load(
    urdf_path,
    mesh_dir=mesh_path,
    load_meshes=True,
    build_scene_graph=True,
    load_collision_meshes=False,
    build_collision_scene_graph=False,
)

viser_robot = ViserUrdf(
    viser_server,
    urdf_or_path=urdf,
    load_meshes=True,
    load_collision_meshes=False,
    mesh_color_override=(0.6, 0.6, 0.6, 1.0),
    collision_mesh_color_override=(1.0, 0.0, 0.0, 0.5),
)

viser_server.scene.add_box(
    name="/table",
    dimensions=(0.9, 1.4, 0.08),
    color=(0.76, 0.60, 0.42),
    position=(0, 0, table_height - 0.08 / 2),
    receive_shadow=True,
    cast_shadow=True,
)

directional_light = viser_server.scene.add_light_directional(
        name="/control0/directional_light",
        color=(186, 219, 173),
        cast_shadow=True,
)


viser_robot.update_cfg(default_q)



: 

Viser functions to visualize spheres and contact forces

In [25]:
vis_spheres = {} 
def visualize_spheres(contact_sphere_params):
    for key, contact_sphere in contact_sphere_params.items():
        if key in vis_spheres:
            sphere = vis_spheres[key]
            sphere.position = contact_sphere['center']
            sphere.radius = contact_sphere['radius']
        else:
            sphere_name = key
            center = contact_sphere['center']
            radius = contact_sphere['radius']
            color = contact_sphere.get('color', (1.0, 0.0, 0.0))
            vis_spheres[key]  = viser_server.scene.add_icosphere(
                name=sphere_name,
                position=center,
                radius=radius,
                color=color,
                cast_shadow=True,
                receive_shadow=True,
                opacity=0.5
            )


def visualize_contact_force(contact_pos, contact_forces):
    #line segment to represent the force arrow 
    points = []
    for i, pos in enumerate(contact_pos):
        force = contact_forces[i]
        force_magnitude = np.linalg.norm(force)
        if force_magnitude > 1e-6:
            force_dir = force / force_magnitude
            arrow_length = min(force_magnitude * 0.05, 1.0)  # scale for visualization
            arrow_start = pos
            arrow_end = pos + force_dir * arrow_length
            points.append([arrow_start, arrow_end])
    if len(points) > 0:
        viser_server.scene.add_line_segments(
                name=f"/contact_forces",
                points=points,
                line_width=2,
                colors=(0.0, 0.0, 1.0),
        )
        
            

            

### 3. 

Use Pinocchio to compute the Jacobians for the robot given the URDF and the joint configuration.

In [26]:
import pinocchio as pin
from pinocchio import FrameType, ReferenceFrame

pin_robot = pin.RobotWrapper.BuildFromURDF(urdf_path, [mesh_path])
pin_model = pin_robot.model 
pin_data = pin_robot.data

In [27]:
import copy

def get_fk_link_poses(q, pin_model, pin_data, link_names): 
    pin.forwardKinematics(pin_model, pin_data, q)
    pin.updateFramePlacements(pin_model, pin_data)

    link_poses = [] 
    for link_name in link_names: 
        for frame_id, frame in enumerate(pin_model.frames): 
            if frame.type == FrameType.BODY and frame.name == link_name:
                ee_frame_id = frame_id
                break 
        link_pose = pin_data.oMf[ee_frame_id]
        link_pose_pos = link_pose.translation
        link_pose_rot = link_pose.rotation
        link_pose_mat = np.eye(4)
        link_pose_mat[:3, :3] = link_pose_rot
        link_pose_mat[:3, 3] = link_pose_pos
        link_poses.append(copy.deepcopy(link_pose_mat))
    
    return copy.deepcopy(link_poses)

def get_link_jacobians(q, pin_model, pin_data, link_names): 
    pin.computeJointJacobians(pin_model, pin_data, q)
    J_list = []

    for link_name in link_names: 
        for frame_id, frame in enumerate(pin_model.frames): 
            if frame.type == FrameType.BODY and frame.name == link_name:
                ee_frame_id = frame_id
                break 
        J = pin.getFrameJacobian(pin_model, pin_data, ee_frame_id, ReferenceFrame.LOCAL_WORLD_ALIGNED) #(JR, Jp)
        
        J = np.vstack((J[3:, :], J[:3, :]))
        J_list.append(copy.deepcopy(J))

    return copy.deepcopy(J_list)



### 4.

This function will compute the collision spheres for the robot. We will start by placing one at each fingertip. Furthermore, we will return the jacobians for each link and the position of the link positions to be used later to calculate the contact Jacobians.



In [28]:
from utils.so3 import * 

#generate a list of contact spheres that will be referenced relative to the link frames, for example a thumb tip link can have one or more contact spheres defined in its frame
def compute_contact_spheres(link_poses, link_jacobians):
    """
    return: 
    contact_sphere_vec: (num_spheres, 4) list of contact spheres where radius is last index
    link_J_for_spheres: the jacobian for the link that each sphere is attached to 
    link_centers_for_spheres: the center of the link that each sphere is attached to 
    contact_sphere_params: dictionary describing the spheres {(link): {(center):(3,), (radius):(float)}}
    """
        
    #generate contact spheres for collisions, 
    contact_sphere_params = {}

    thumb4_pose = link_poses[2]
    thumb4_J = link_jacobians[2]
    contact_sphere_params['thumb_1'] = {"center": (thumb4_pose[:3, 3] + 0.027*thumb4_pose[:3, 2] - 0.001*thumb4_pose[:3, 0]), "radius": 0.015} 

    ######################################################################
    index4_pose = link_poses[4]
    index4_J = link_jacobians[4]
    contact_sphere_params['index_1'] = {"center": (index4_pose[:3, 3] + 0.027*index4_pose[:3, 2] - 0.001*index4_pose[:3, 0]), "radius": 0.015} 

    middle4_pose = link_poses[6]
    middle4_J = link_jacobians[6]
    contact_sphere_params['middle_1'] = {"center": (middle4_pose[:3, 3] + 0.027*middle4_pose[:3, 2] - 0.001*middle4_pose[:3, 0]), "radius": 0.015}

    ring4_pose = link_poses[8]
    ring4_J = link_jacobians[8]
    contact_sphere_params['ring_1'] = {"center": (ring4_pose[:3, 3] + 0.027*ring4_pose[:3, 2] - 0.001*ring4_pose[:3, 0]), "radius": 0.015}

    little4_pose = link_poses[10]
    little4_J = link_jacobians[10]
    contact_sphere_params['little_1'] = {"center": (little4_pose[:3, 3] + 0.027*little4_pose[:3, 2] - 0.001*little4_pose[:3, 0]), "radius": 0.015}
    ######################################################################

    contact_sphere_vec = [] 
    for key in contact_sphere_params.keys(): 
        contact_sphere_vec.append(np.hstack((contact_sphere_params[key]['center'], contact_sphere_params[key]['radius'])))

    contact_sphere_vec = np.array(contact_sphere_vec)  #(num_spheres, 4)

    link_J_for_spheres = np.concatenate(
        [
            thumb4_J,
            index4_J,
            middle4_J,
            ring4_J,
            little4_J
        ]
    )
    link_J_for_spheres = link_J_for_spheres.reshape(len(contact_sphere_params), 6, n_joints)

    link_centers_for_sphere = np.concatenate(
        [
            thumb4_pose[:3, 3].reshape(1, 3),
            index4_pose[:3, 3].reshape(1, 3),
            middle4_pose[:3, 3].reshape(1, 3),
            ring4_pose[:3, 3].reshape(1, 3),
            little4_pose[:3, 3].reshape(1, 3),
        ]
    ) #(num_spheres, 3)

    return contact_sphere_vec, link_J_for_spheres, link_centers_for_sphere, contact_sphere_params



### 5. 

This function will compute contacts between spheres, the signed distance function, and the contact jacobian for the robot and the object in the scene given the contact points.

In [29]:
def compute_contact_states(contact_sphere_vec, link_centers_for_sphere, link_J_for_spheres, q_o, object_radius, table_height, threshold_phi=0.001):
    """
    return: 
    contact_pos: (N+1, 3)
    phi: sign distance (N+1,)
    contact_flag: (N+1,)
    n_hat: normal vectors (N+1, 1, 3) (object - robot)
    t_hat: tangent vectors (N+1, 2, 3)
    J_o: object contact jacobian (N+1, 3, 3) for each contact how it affects the object  
    J_a: robot contact jacobian (N+1, 3, n_control) for each contact how it affects robot joint velocity
    J_c: combined contact jacobian (N+1, 3, n_dof); [J_o; -J_a]
    """

    N = contact_sphere_vec.shape[0]
    n_control = 21 
    n_dof = n_control + 3 #for object 3 dof 

    robot_sphere_center = contact_sphere_vec[:, :3]  #(N, 3)
    robot_sphere_radius = contact_sphere_vec[:, 3]  #(N,)

    object_sphere_center = q_o.reshape(1, 3)  #(1, 3)
    object_sphere_radius = object_radius #(scalar)

    alpha = object_sphere_center - robot_sphere_center #(N, 3)
    alpha_norm = np.linalg.norm(alpha, axis=1).reshape(N, 1)
    alpha_hat = alpha / (alpha_norm + 1e-8) #(N, 3)

    phi = np.zeros((N+1,), dtype=np.float64)
    phi[0] = object_sphere_center[0, 2] - table_height  - object_sphere_radius  #object-table contact
    phi[1:] = alpha_norm.flatten() - (robot_sphere_radius + object_sphere_radius)  #object-robot contacts

    ######################################################################
    contact_flag = phi < threshold_phi
    ######################################################################

    contact_pos = np.zeros((N+1, 3), dtype=np.float64)
    contact_pos[0, :] = object_sphere_center[0, :] - np.array([0.0, 0.0, object_sphere_radius])  #object-table contact point
    ######################################################################
    contact_pos[1:, :] = robot_sphere_center + (robot_sphere_radius.reshape((-1, 1)).repeat(3, axis=1) * alpha_hat)  #object-robot contact points
    ######################################################################

    n_hat = np.zeros((N+1, 1, 3), dtype=np.float64)
    n_hat[0, 0, :] = np.array([0, 0, 1])
    n_hat[1:, 0, :] = alpha_hat 

    t_hat = np.zeros((N+1, 2, 3), dtype=np.float64)
    #for table
    t_hat[0, 0, :] = np.array([1, 0, 0])
    t_hat[0, 1, :] = np.array([0, 1, 0])

    #contact jacobians 

    #for object contacts: object velocities to object contact velocities 
    J_o = np.zeros((N+1, 3, 6), dtype=np.float64) #(N+1, 3, 6) how object contact force affects object 6dof (well cut the rotation ones out after)
    J_o[0, :, 3:] = np.eye(3)  #object-table contact affects only object linear velocity
    J_o[0, :, :3] = -skew(np.array([0, 0, -1.0]))
    batch_r = contact_pos[1:] - object_sphere_center #(N, 3)
    J_o[1:, :, 3:] = np.eye(3).reshape(1, 3, 3).repeat(N, axis=0)
    J_o[1:, :, :3] = -batch_skew(batch_r)

    #for object-robot contacts
    r = np.ones((N, 3)) #random vectory to do cross product and get one tangent vector 
    t_hat[1:, 0, :] = np.cross(n_hat[1:, 0, :], r)
    t_hat[1:, 0, :] = t_hat[1:, 0, :] / (np.linalg.norm(t_hat[1:, 0, :], axis=1).reshape(N, 1) + 1e-8)
    t_hat[1:, 1, :] = np.cross(n_hat[1:, 0, :], t_hat[1:, 0, :])

    J_o = J_o[:, :, 3:] #(N+1, 3, 3) #ignore the rotation part 
    J_a = np.zeros((N+1, 3, n_control), dtype=np.float64) #(N+1, 3, n_control), first index is 0 for table contact has no impact on joint torques 

    delta = contact_pos[1:] - link_centers_for_sphere #(N, 3)
    #matrix to compute the contact jacobian from the 
    T = np.zeros((N, 3, 6), dtype=np.float64) #(N, 3, 6) Jc = [-[r]x | I] @ [Jw ; Jv]
    T[:, :, :3] = -batch_skew(delta)
    T[:, :, 3:] = np.eye(3).reshape(1, 3, 3).repeat(N, axis=0)
    J_a[1:] = T @ link_J_for_spheres #(N, 3, n_control)

    J_c_temp = np.concatenate([J_o, -J_a], axis=-1) #(N+1, 3, n_dof) #robot and object velocities to contact velocities 
    J_cn = n_hat @ J_c_temp #(N+1, 1, n_dof)
    J_ct = t_hat @ J_c_temp #(N+1, 2, n_dof)
    J_c = np.concatenate([J_cn, J_ct], axis=1) #(N+1, 3, n_dof)

    return contact_pos, phi, contact_flag, n_hat, t_hat, J_c


Now to visualize what we have so far, you should be able to move around the objects in the Mujoco viewer and see the scene replicated in the web in Viser. You should also see the red contact spheres on the fingertips as well as the green ball.

![Viser view](assets/images/viser_image.png)

In [31]:
model = mj.MjModel.from_xml_path(env_path)
data = mj.MjData(model)
viewer = mjv.launch_passive(model, data)

while viewer.is_running(): 
    #q_a is (21)
    #q_o is (3) xyz of sphere (ignoring rotation of the sphere)

    q_a = data.qpos[:n_joints].copy()
    q_o = data.qpos[n_joints:n_joints+3].copy()
    object_radius = model.geom_size[mj.mj_name2id(model, mj.mjtObj.mjOBJ_GEOM, "object_geom")][0]

    viser_robot.update_cfg(q_a)

    list_link_names = [
                        "palm",
                        "finger_r_link_1_thumb3",
                        "finger_r_link_1_thumb4",
                        "finger_r_link_2_index3",
                        "finger_r_link_2_index4",
                        "finger_r_link_3_middle3",
                        "finger_r_link_3_middle4",
                        "finger_r_link_4_ring3",
                        "finger_r_link_4_ring4",
                        "finger_r_link_5_little3",
                        "finger_r_link_5_little4",
                    ]

    link_poses = get_fk_link_poses(q_a, pin_model, pin_data, list_link_names) #(11, 4, 4)
    link_jacobians = get_link_jacobians(q_a, pin_model, pin_data, list_link_names) #(11, 6, 21)

    contact_sphere_vec, link_J_for_spheres, link_centers_for_sphere, contact_sphere_params = compute_contact_spheres(link_poses, link_jacobians)
    visualize_spheres(contact_sphere_params)
    visualize_spheres(
        {
            "object_sphere": {
                "center": q_o,
                "radius": object_radius,
                "color": (0.0, 1.0, 0.0),
            }
        }
    )

    mj.mj_step(model, data)
    viewer.sync()

(viser) Connection opened (0, 1 total), 156 persistent messages

### 6.  

Now we will compute the kinodynamics matrices used in the optimization using the following formulas: 
where
\begin{align}
\mathbf{P}(q) &:= 
\begin{bmatrix}
\epsilon \mathbf{M}_o(q) / h^2 & 0 \\
0 & \mathbf{K}_a
\end{bmatrix} \tag{3b} \\
\mathbf{b}(q, u) &:= -
\begin{bmatrix}
\epsilon \mathbf{M}_o(q) q^o / h^2 + \tau^o \\
\mathbf{K}_a u + \tau^a
\end{bmatrix}
\tag{3c}
\end{align}

\begin{equation}
\mathbf{c}_i(q) := [\phi_i(q), 0, 0]^\top - \mathbf{J}_i(q) q
\end{equation}

In [33]:
def compute_kinodynamic_matrices(q_o, q_a, u, contact_pos, phi, contact_flag, J_c):
    K_a = [1.0 for _ in range(len(q_a))]  #(n_control,)
    K_a[0] = 100.0
    K_a = np.array(K_a, dtype=np.float64)  #(n_control,)

    M_o = [0.52, 0.52, 0.52] 
    M_o = np.array(M_o, dtype=np.float64)  #(3,)
    eps = 0.1 
    h = 0.01 

    contact_pos = contact_pos[contact_flag] 
    phi = phi[contact_flag]
    J_c = J_c[contact_flag]

    P_diag = np.zeros(len(q_a) + len(q_o), dtype=np.float64) #(n_dof,)
    P_diag[:len(q_o)] = eps * M_o / h**2 
    P_diag[len(q_o):] = K_a

    b = np.zeros(len(q_a) + len(q_o), dtype=np.float64) #(n_dof,)
    tau_o = -M_o * np.array([0, 0, 9.81]) #(3,)
    b[:len(q_o)] = -((eps * M_o * q_o / h**2) + tau_o)
    tau_a = np.zeros((len(q_a),), dtype=np.float64)
    b[len(q_o):] = -(K_a * u + tau_a)

    c = -J_c @ np.hstack([q_o, q_a]) #(num_contacts, 3)
    c[:, 0] += phi 

    return P_diag, b, J_c, c 

### 7.

Finally we will use the interior point method (IPOPT) from Casadi to solve the optimization problem.

\begin{equation}
\min_{q_+} \;
\frac{1}{2} q_+^\top \mathbf{P} q_+ + \mathbf{b}^\top q_+
- \frac{1}{\kappa} \sum_{i=1}^{n_c} 
\log \left( \frac{\nu_{n_i}^2}{\mu_i^2} - \| \nu_{t_i} \|^2 \right),
\tag{10}
\end{equation}

where $\nu_i = \mathbf{J}_i q_+ + \mathbf{c}_i$, and 
$\nu_i = (\nu_{n_i}, \nu_{t_i})$ with 
$\nu_{n_i} \in \mathbb{R}$ and 
$\nu_{t_i} \in \mathbb{R}^2$. 
The resulting dynamics creates a force-field effect where bodies that are not in contact apply forces inversely proportional to their distance.

Furthermore, the following equation gives the average contact force over the timestep:

\begin{equation}
\lambda_{\kappa,i} =
\frac{2 \kappa^{-1}}{\nu_{n_i}^2 / \mu_i^2 - \| \nu_{t_i} \|^2}
\begin{bmatrix}
\nu_{n_i} / \mu_i^2 \\
-\nu_{t_i}
\end{bmatrix}.
\tag{12}
\end{equation}


In [34]:




from casadi import (
    SX,
    DM,
    dot,
    mtimes,
    reshape,
    log,
    sum1,
    nlpsol,
    hcat,
    sum2,
    vertcat,
    sqrt,
)

def forward_dynamics(q_o, q_a, J_c_active, P_diag, b, c):
    mu = 0.5 
    kappa = 100.0 

    init_guess = np.hstack([q_o, q_a])

    n = P_diag.shape[0]
    nc = J_c_active.shape[0]

    #ipopt 
    x = SX.sym("x", n) #(n,)

    #numpy array to CasADi constants 
    #J_c: (nc, 3, n) -> (3*nc, n)

    J_c_dm = DM(J_c_active.reshape(-1, n)) #(3*nc, n)

    #P_diag, b: (n, ) -> (n,1)
    P_diag_dm = DM(P_diag.reshape(-1, 1)) #(n,1)
    b_dm = DM(b.reshape(-1, 1)) #(n,1)

    #c: (nc, 3) -> (3*nc, 1)
    c_dm = DM(c.reshape(-1, 1)) #(3*nc, 1)

    quad = 0.5 * dot(P_diag_dm, x**2) #scalar 
    lin = dot(b_dm, x) #scalar 

    #contact velocities 
    nu_flat = mtimes(J_c_dm, x) + c_dm #(3*nc, 1)

    nu = reshape(nu_flat, 3, nc).T #(nc, 3): [nu_n, nu_tx, nu_ty] 

    nu_n = nu[:, 0] #(nc,)
    nu_tx = nu[:, 1] #(nc,)
    nu_ty = nu[:, 2] #(nc,)

    nu_t_sq = nu_tx**2 + nu_ty**2 #+(1e-8) #(nc,)

    #arg for log barrier 
    arg = nu_n**2 / (mu**2) - nu_t_sq #(nc,)
    log_barrier = -(1.0/kappa) * sum1(log(arg)) #scalar 

    f = quad + lin + log_barrier 

    #nu_n >= 0 
    g = vertcat(nu_n, arg) # (3*nc,)

    nlp = {"x": x, "f": f, "g": g}

    opts = {
        "ipopt.print_level": 0,
        "print_time": 0,
        "ipopt.sb": "yes",
        "ipopt":{
            "nlp_scaling_method": "gradient-based",
            "tol": 1e-8, 
            "warm_start_bound_push": 1e-10,
        }
    }

    solver = nlpsol("solver", "ipopt", nlp, opts)
    x0 = init_guess 
    lbg = DM.zeros(g.shape[0])
    ubg = DM.inf(g.shape[0])

    try: 
        sol = solver(x0=x0, lbg=lbg, ubg=ubg)
        info = solver.stats() 

        if info["success"]:
            q_plus = sol["x"].toarray()
        else:
            print("Solver failed to find a solution")
            q_plus = init_guess.reshape(-1, 1)
    except Exception as e:
        print("Solver exception:", e)
        q_plus = init_guess.reshape(-1, 1)

    def compute_lambda(q_plus, J_c_active, c, kappa, mu):
        nu = (J_c_active @ q_plus).reshape(-1, 3) + c #(nc, 3)
        nu_n = nu[:, 0] #(nc,)
        nu_t = nu[:, 1:] #(nc,2)
        nu_t_sq = nu_t**2 
        arg = nu_n**2 / (mu**2) - nu_t_sq.sum(axis=-1) #(nc,)
        coeff = 2*1/kappa * (1/arg).reshape(-1, 1)
        lambda_ = coeff * np.concatenate(
            [nu_n.reshape(-1, 1)/mu**2, -nu_t], axis=-1
        )
        return lambda_ #(nc, 3)


    lambda_plus = compute_lambda(q_plus, J_c_active, c, kappa, mu)

    return q_plus.reshape(-1), lambda_plus

(viser) Connection closed (0, 0 total)

### Putting it all together

Now running this you should see the contact forces as blue lines appearing when two collision bodies come into contact. 

In [22]:
# Load model and data
model = mj.MjModel.from_xml_path(env_path)
data = mj.MjData(model)
viewer = mjv.launch_passive(model, data)

while viewer.is_running(): 
    #q_a is (21)
    #q_o is (3) xyz of sphere (ignoring rotation of the sphere)

    q_a = data.qpos[:n_joints].copy()
    q_o = data.qpos[n_joints:n_joints+3].copy()
    object_radius = model.geom_size[mj.mj_name2id(model, mj.mjtObj.mjOBJ_GEOM, "object_geom")][0]

    viser_robot.update_cfg(q_a)

    list_link_names = [
                        "palm",
                        "finger_r_link_1_thumb3",
                        "finger_r_link_1_thumb4",
                        "finger_r_link_2_index3",
                        "finger_r_link_2_index4",
                        "finger_r_link_3_middle3",
                        "finger_r_link_3_middle4",
                        "finger_r_link_4_ring3",
                        "finger_r_link_4_ring4",
                        "finger_r_link_5_little3",
                        "finger_r_link_5_little4",
                    ]

    link_poses = get_fk_link_poses(q_a, pin_model, pin_data, list_link_names) #(11, 4, 4)
    link_jacobians = get_link_jacobians(q_a, pin_model, pin_data, list_link_names) #(11, 6, 21)

    contact_sphere_vec, link_J_for_spheres, link_centers_for_sphere, contact_sphere_params = compute_contact_spheres(link_poses, link_jacobians)
    visualize_spheres(contact_sphere_params)
    visualize_spheres(
        {
            "object_sphere": {
                "center": q_o,
                "radius": object_radius,
                "color": (0.0, 1.0, 0.0),
            }
        }
    )
    contact_pos, phi, contact_flag, n_hat, t_hat, J_c = compute_contact_states(contact_sphere_vec, link_centers_for_sphere, link_J_for_spheres, q_o, object_radius, table_height)

    u = q_a  
    # u[0] = -0.01 
    P_diag, b, J_c_active, c = compute_kinodynamic_matrices(q_o, q_a, u, contact_pos, phi, contact_flag, J_c)

    q_plus, lambda_plus = forward_dynamics(q_o, q_a, J_c_active, P_diag, b, c)

    contact_forces = (
        lambda_plus[:, 0:1] * n_hat[contact_flag, 0, :]
        + lambda_plus[:, 1:2] * t_hat[contact_flag, 0, :]
        + lambda_plus[:, 2:3] * t_hat[contact_flag, 1, :]
    )

    visualize_contact_force(contact_pos[contact_flag], contact_forces)

    mj.mj_step(model, data)
    viewer.sync()

CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:408]
CasADi - 2025-11-12 16:17:13 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).")